In [4]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
!unzip crawl-300d-2M-subword.zip

--2020-05-24 07:55:20--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/crawl-300d-2M-subword.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 2606:4700:10::6816:4b8e, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5828358084 (5.4G) [application/zip]
Saving to: ‘crawl-300d-2M-subword.zip’

100%[====================================>] 5,828,358,084 13.4MB/s   in 7m 9s  

2020-05-24 08:02:30 (12.9 MB/s) - ‘crawl-300d-2M-subword.zip’ saved [5828358084/5828358084]

Archive:  crawl-300d-2M-subword.zip
  inflating: crawl-300d-2M-subword.vec  
  inflating: crawl-300d-2M-subword.bin  


In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath(''))

In [6]:



import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import torch.nn as nn
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from importlib import reload
import facebook_hateful_memes_detector
reload(facebook_hateful_memes_detector)

from facebook_hateful_memes_detector import read_json_lines_into_df
from facebook_hateful_memes_detector import FasttextPooledModel
from facebook_hateful_memes_detector.preprocessing import TextImageDataset




<module 'facebook_hateful_memes_detector' from '/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/__init__.py'>

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
dev = read_json_lines_into_df('../data/dev.jsonl')
train = read_json_lines_into_df('../data/train.jsonl')
test = read_json_lines_into_df('../data/test.jsonl')

train_text = list(train.text)
dev_text = list(dev.text)
test_text = list(test.text)

train_img = list(map(lambda l: "../data/"+l,train.img))
test_img = list(map(lambda l: "../data/"+l,test.img))
dev_img = list(map(lambda l: "../data/"+l,dev.img))

# from sklearn.preprocessing import OneHotEncoder
# enc = OneHotEncoder(categories=[[0, 1]],handle_unknown='error', sparse=False)
train_labels = torch.tensor(train.label)
dev_labels = torch.tensor(dev.label)

train_text = np.array(train_text)
dev_text = np.array(dev_text)
test_text = np.array(test_text)

train_img = np.array(train_img)
test_img = np.array(test_img)
dev_img = np.array(dev_img)


In [9]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


def my_collate(batch):
    text = [item[0] for item in batch]
    image = torch.stack([item[1] for item in batch])
    label = [item[2] for item in batch]
    label = torch.LongTensor(label) if label[0] is not None else label
    return [text, image, label]

In [ ]:
from PIL import Image
channel3 = [np.array(Image.open(im)).shape[2] == 3 for im in train_img]
all(channel3)

from tqdm import tqdm_notebook as tqdm, trange
training_fold_dataset = TextImageDataset(train_text, train_img, train_labels, image_transform=preprocess)
testing_fold_dataset = TextImageDataset(test_text, test_img, image_transform=preprocess)
train_loader = DataLoader(training_fold_dataset, batch_size=64, collate_fn=my_collate, 
                          shuffle=False,num_workers=8, pin_memory=True)



print("Start Iterating")
epochs = 10
for epoch in tqdm_notebook(range(epochs)):
    train_loss = 0
    val_loss = 0
    accuracy = 0
    print("Epoch = ", epoch)

    # Training the model
    counter = 0
    for i, (texts, images, labels) in enumerate(tqdm_notebook(train_loader)):
        print("Epoch = ", epoch, "Sub = ", i)



In [ ]:
from sklearn.model_selection import KFold
from tqdm import tqdm_notebook as tqdm, trange
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, precision_recall_fscore_support

n_tests = 1
for _ in range(n_tests):
    dataset = TextImageDataset(train_text, train_img, train_labels, image_transform=preprocess)
    size = len(dataset)
    training_fold_dataset, testing_fold_dataset = torch.utils.data.random_split(dataset, [int(size*0.8), size - int(size*0.8)])
    
    train_loader = DataLoader(training_fold_dataset, batch_size=512, collate_fn=my_collate, 
                              shuffle=True,num_workers=32, pin_memory=True)
    val_loader = DataLoader(testing_fold_dataset, batch_size=512, collate_fn=my_collate, 
                            shuffle=True,num_workers=32, pin_memory=True)
    
    model = FasttextPooledModel(300, 1024, 2, "crawl-300d-2M-subword.bin",)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    
    epochs = 5
    try:
        with tqdm(list(range(epochs))) as epo:
            for epoch in epo:
                train_loss = 0
                val_loss = 0
                accuracy = 0

                # Training the model
                _ = model.train()
                counter = 0
                with tqdm(train_loader) as data_batch:
                    for texts, images, labels in data_batch:
                        optimizer.zero_grad()
                        logits, loss = model.forward(texts, images, labels)
                        loss.backward()
                        optimizer.step()
                        train_loss += loss.item()*labels.size(0)


    except (KeyboardInterrupt, Exception) as e:
        epochs.close()
        data_batch.close()
        raise
    epo.close()
    data_batch.close()
    
    _ = model.eval()
    labels_list = []
    proba_list = []
    predictions_list = []
    with torch.no_grad():

        for texts, images, labels in val_loader:
            logits, valloss = model.forward(texts, images, labels)
            logits = torch.softmax(logits, dim=1)
            top_p, top_class = logits.topk(1, dim=1)
            labels = list(map(lambda x: x.item() ,list(labels)))
            labels_list.extend(labels)
            top_class = top_class.flatten()
            top_class = list(map(lambda x: x.item() ,list(top_class)))
            probas = list(map(lambda x: x.item() ,list(logits[:,1])))
            predictions_list.extend(list(top_class))
            proba_list.extend(probas)
    auc = roc_auc_score(labels_list, proba_list)
    p_micro, r_micro, f1_micro, _ = precision_recall_fscore_support(labels_list, predictions_list, average="micro")
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(labels_list, predictions_list, average="macro")
    p_weighted, r_weighted, f1_weighted, _ = precision_recall_fscore_support(labels_list, predictions_list, average="weighted")
    validation_scores = [p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, p_weighted, r_weighted, f1_weighted, auc]
    
    #
    labels_list = []
    proba_list = []
    predictions_list = []
    with torch.no_grad():

        for texts, images, labels in train_loader:
            logits, valloss = model.forward(texts, images, labels)
            logits = torch.softmax(logits, dim=1)
            top_p, top_class = logits.topk(1, dim=1)
            labels = list(map(lambda x: x.item() ,list(labels)))
            labels_list.extend(labels)
            top_class = top_class.flatten()
            top_class = list(map(lambda x: x.item() ,list(top_class)))
            probas = list(map(lambda x: x.item() ,list(logits[:,1])))
            predictions_list.extend(list(top_class))
            proba_list.extend(probas)
    auc = roc_auc_score(labels_list, proba_list)
    p_micro, r_micro, f1_micro, _ = precision_recall_fscore_support(labels_list, predictions_list, average="micro")
    p_macro, r_macro, f1_macro, _ = precision_recall_fscore_support(labels_list, predictions_list, average="macro")
    p_weighted, r_weighted, f1_weighted, _ = precision_recall_fscore_support(labels_list, predictions_list, average="weighted")
    train_scores = [p_micro, r_micro, f1_micro, p_macro, r_macro, f1_macro, p_weighted, r_weighted, f1_weighted, auc]
    index = ["p_micro", "r_micro", "f1_micro", "p_macro", "r_macro", "f1_macro", "p_weighted", "r_weighted", "f1_weighted", "auc"]
    pd.DataFrame(data=dict(train=train_scores, val=validation_scores), index=index)
    

In [34]:
test_dataset = TextImageDataset(test_text, test_img, None, image_transform=preprocess)
test_loader = DataLoader(test_dataset, batch_size=1024, collate_fn=my_collate, 
                              shuffle=False,num_workers=32, pin_memory=False)

proba_list = []
predictions_list = []
with torch.no_grad():
    for texts, images, labels in test_loader:
        logits, valloss = model.forward(texts, images, labels)
        logits = torch.softmax(logits, dim=1)
        top_p, top_class = logits.topk(1, dim=1)
        top_class = top_class.flatten()
        top_class = list(map(lambda x: x.item() ,list(top_class)))
        probas = list(map(lambda x: x.item() ,list(logits[:,1])))
        predictions_list.extend(list(top_class))
        proba_list.extend(probas)

/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/models/text_models/FasttextPooled.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [35]:
submission = pd.DataFrame(dict(id=test.id, proba=proba_list, label=predictions_list), columns=["id", "proba", "label"])
submission.to_csv("submission.csv",index=False)

In [36]:
submission.head()

,id,proba,label
0,16395,0.656999,1
1,37405,0.398898,0
2,94180,0.282765,0
3,54321,0.467119,0
4,97015,0.589054,1
